In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import zscore


def read_user_id():
    with open('./input.txt', 'r') as f:
        return [l.strip().split(',') for l in  f.readlines()]


def write_output(prediction):
    with open('./output.txt', 'w') as f:
        for pred in prediction:
            f.write(pred+"\n")


def do(inputs):
    string_results = []
    for user, movie in inputs:
        key_user = int(user)
        key_movie = int(movie)
#         print("(user, movie) = (%d, %d)"%(key_user, key_movie))

#         string_results.append(task2(key_user, key_movie))

    return string_results


def initialize_train_data():
#     global movieIds, userIds
    df_train = pd.read_csv('data/ratings_train.csv', usecols = ['userId', 'movieId', 'rating'])
    return df_train
#     movieIds = sorted(trainset['movieId'].unique())
#     userIds = sorted(trainset['userId'].unique())
    

def initialize_test_data():
    targets = pd.read_csv('data/ratings_test.csv', usecols = ['userId', 'movieId', 'rating'])
    return targets


def preprocess_by_user():
    df_train.groupby(['userId']).rating.transform(lambda x : zscore(x,ddof=1)).head(3)


def RMSE(predictions, targets):
    return np.sqrt(((predictions - targets)**2).mean())




if __name__ == "__main__":
    global df_train
    df_train = initialize_train_data()
    df_valid = initialize_valid_data()
    Mean = df_train.groupby(['userId']).rating.transform('mean')
    Std = df_train.groupby(['userId']).rating.transform('std') 
    df_train['rating']=df_train.groupby(['userId']).rating.transform(lambda x : zscore(x, ddof=1))
    movieIds = sorted(df_train['movieId'].unique())
    userIds = sorted(df_train['userId'].unique())
    df_ui = pd.DataFrame(index=sorted(movieIds), columns=sorted(userIds) )
    for index, rows in  df_train.iterrows():
         df_ui.loc[rows['movieId']][rows['userId']] = rows['rating']
            
        
    
#     df_svd = build_svd(df_train)
#     # execute
#     user_ids = read_user_id()
#     result = do(user_ids)
#     write_output(result)
      
#     

In [3]:

def build_svd(model, K=100):
    filled_model= model.apply(lambda row: row.fillna(row.mean()), axis=1)
    u, s, vh = np.linalg.svd(filled_model, True)
    u = u[:,:K]
    Sigma = np.diag(s[:K])
    vh = vh[:K, :]
    user_factors =  np.matmul(u, np.sqrt(Sigma))
    item_factors =  np.matmul(np.sqrt(Sigma),vh)
    df_prediction = pd.DataFrame(np.matmul(user_factors, item_factors),index=model.index, columns=model.columns)
    df_prediction = df_prediction.round(4)
    return df_prediction


In [4]:
decomposition = build_svd(df_ui.fillna(0))

In [5]:
# predict with svd 
# decomposition[int(uid)]
decomposition[1]

1        0.0035
2       -0.0085
3        0.0434
4       -0.0047
5        0.0039
          ...  
83322   -0.0026
83359   -0.0051
83361   -0.0013
83411   -0.0051
83603   -0.0013
Name: 1, Length: 6901, dtype: float64

In [12]:
movieIds = sorted(df_train['movieId'].unique())
userIds = sorted(df_train['userId'].unique())
len(movieIds), len(userIds)

(6901, 515)

In [7]:
class ShuffleIterator(object):
    """
    Randomly generate batches
    """
    def __init__(self, inputs, batch_size=10):
        self.inputs = inputs
        self.batch_size = batch_size
        self.num_cols = len(self.inputs)
        self.len = len(self.inputs[0])
        self.inputs = np.transpose(np.vstack([np.array(self.inputs[i]) for i in range(self.num_cols)]))

    def __len__(self):
        return self.len

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def next(self):
        ids = np.random.randint(0, self.len, (self.batch_size,))
        out = self.inputs[ids, :]
        return [out[:, i] for i in range(self.num_cols)]

In [8]:
class OneEpochIterator(ShuffleIterator):
    """
    Sequentially generate one-epoch batches, typically for test data
    """
    def __init__(self, inputs, batch_size=10):
        super(OneEpochIterator, self).__init__(inputs, batch_size=batch_size)
        if batch_size > 0:
            self.idx_group = np.array_split(np.arange(self.len), np.ceil(self.len / batch_size))
        else:
            self.idx_group = [np.arange(self.len)]
        self.group_id = 0

    def next(self):
        if self.group_id >= len(self.idx_group):
            self.group_id = 0
            raise StopIteration
        out = self.inputs[self.idx_group[self.group_id], :]
        self.group_id += 1
        return [out[:, i] for i in range(self.num_cols)]

In [ ]:
import tensorflow 

BATCH_SIZE = 1024
USER_NUM = 515
ITEM_NUM = 6901
DIM = 15
EPOCH_MAX = 100
DEVICE = "/cpu:0"


def clip(x):
    return np.clip(x, 1.0, 5.0)


def make_scalar_summary(name, val):
    return summary_pb2.Summary(value=[summary_pb2.Summary.Value(tag=name, simple_value=val)])


def build_deep(df_train):
    samples_per_batch = df_train.shape[0] // BATCH_SIZE

    iter_train = ShuffleIterator([df_train["userId"],
                                         df_train["movieId"],
                                         df_train["rating"]],
                                        batch_size=BATCH_SIZE)

    iter_valid = OneEpochIterator([df_valid["userId"],
                                         df_valid["movieId"],
                                         df_valid["rating"]],
                                        batch_size=-1)

    use = tf.placeholder(tf.int32, shape=[None], name="id_user")
    item = tf.placeholder(tf.int32, shape=[None], name="id_item")
    rate = tf.placeholder(tf.float32, shape=[None])

    infer, regularizer = ops.inference_svd(user_batch, item_batch, user_num=USER_NUM, item_num=ITEM_NUM, dim=DIM,device=DEVICE)
    global_step = tf.contrib.framework.get_or_create_global_step()
    _, train_op = ops.optimization(infer, regularizer, rate_batch, learning_rate=0.001, reg=0.05, device=DEVICE)

    init_op = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_op)
        summary_writer = tf.summary.FileWriter(logdir="./log", graph=sess.graph)
        print("{} {} {} {}".format("epoch", "train_error", "val_error", "elapsed_time"))
        errors = deque(maxlen=samples_per_batch)
        start = time.time()
        for i in range(EPOCH_MAX * samples_per_batch):
            users, items, rates = next(iter_train)
            _, pred_batch = sess.run([train_op, infer], feed_dict={user_batch: users,
                                                                   item_batch: items,
                                                                   rate_batch: rates})
            pred_batch = clip(pred_batch)
            errors.append(np.power(pred_batch - rates, 2))
            if i % samples_per_batch == 0:
                train_err = np.sqrt(np.mean(errors))
                test_err2 = np.array([])
                for users, items, rates in iter_test:
                    pred_batch = sess.run(infer, feed_dict={user_batch: users,
                                                            item_batch: items})
                    pred_batch = clip(pred_batch)
                    test_err2 = np.append(test_err2, np.power(pred_batch - rates, 2))
                end = time.time()
                test_err = np.sqrt(np.mean(test_err2))
                print("{:3d} {:f} {:f} {:f}(s)".format(i // samples_per_batch, train_err, test_err,
                                                       end - start))
                train_err_summary = make_scalar_summary("training_error", train_err)
                test_err_summary = make_scalar_summary("test_error", test_err)
                summary_writer.add_summary(train_err_summary, i)
                summary_writer.add_summary(test_err_summary, i)
                start = end